Data Date Range:
    <br />Start time: 2016-05-05 09:40:26
    <br />End time: 2018-01-17 01:59:42

#### This notebook generates the following table:
- recs_viewed_sessionid: recommended trailers that are viewed, and which session (of a login) it is viewed

In [1]:
import pandas as pd
import numpy as np

In [22]:
action_data = pd.read_csv('../Clean_Data/log_trailer_actions_sessionIds.csv')

In [23]:
action_data.head()

,Unnamed: 0,userId,loginId,movieId,action,tstamp,position,sessionIds
0,0,276159,008C57f,118985,PlayedFromLaunch,2016-09-12 13:53:04,0,1
1,1,276159,008C57f,118985,TrailerModalLaunched,2016-09-12 13:53:04,0,2
2,2,276159,008C57f,118985,TrailerModalShutdown,2016-09-12 13:55:31,0,2
3,3,276159,008C57f,157140,TrailerModalLaunched,2016-09-12 13:55:52,0,3
4,4,276159,008C57f,157140,PlayedFromLaunch,2016-09-12 13:55:53,0,3


### Load the dataset with algorithm assignment in each session in the experiment

The variable 'newassignment' stores information about the algorithm assignment of each login session, and is used in many sections, so we recommend you running the block below before running other sections. 

In [24]:
# Get the algorithm assignment
expt_trailer = pd.read_csv('../Data/expt_trailer.csv', sep=',')
alg_assignment = expt_trailer[['userId','loginId','recommenderId']]
alg_assignment.sort_values(by='recommenderId').shape


(510939, 3)

### Generate a table recommendation played, with algorithm assignment

If an action is among: "PlayedFromNext", "PlayedFromRecommendation", "PlayedFromPrevious", it means a recommendation is played, so we keep this row in the table.

In [25]:
# Trailers recommended by an algorithm and played, excluding 'PlayedFromYoutubePlayer'
rec_played = action_data[action_data['action'].isin(["PlayedFromNext", "PlayedFromRecommendation", "PlayedFromPrevious"])]

# Join the table of trailers recommended and played with the table of algorithm assignment
rec_played_alg = pd.merge(rec_played, alg_assignment, on=['loginId','userId'], how='left')
rec_played_alg.head()

,Unnamed: 0,userId,loginId,movieId,action,tstamp,position,sessionIds,recommenderId
0,34,304211,00bSwKf,159193,PlayedFromRecommendation,2017-04-29 07:07:09,4,3,FilmReleaseDate
1,66,304211,00bSwKf,132800,PlayedFromRecommendation,2017-04-29 07:15:57,6,6,FilmReleaseDate
2,70,304211,00bSwKf,161966,PlayedFromRecommendation,2017-04-29 07:18:04,2,6,FilmReleaseDate
3,83,304211,00bSwKf,5617,PlayedFromRecommendation,2017-04-29 07:18:16,0,6,FilmReleaseDate
4,87,304211,00bSwKf,113829,PlayedFromRecommendation,2017-04-29 07:18:34,7,6,FilmReleaseDate


In [26]:
# Fill in NaN values with the default value FilmReleaseDate
# rec_played_alg['recommenderId'].fillna('FilmReleaseDate', inplace=True)

# Or drop na values
rec_played_alg2 = rec_played_alg.dropna(subset=['recommenderId']).copy()
rec_played_alg2.drop("Unnamed: 0",axis=1, inplace=True)
rec_played_alg2.head()

,userId,loginId,movieId,action,tstamp,position,sessionIds,recommenderId
0,304211,00bSwKf,159193,PlayedFromRecommendation,2017-04-29 07:07:09,4,3,FilmReleaseDate
1,304211,00bSwKf,132800,PlayedFromRecommendation,2017-04-29 07:15:57,6,6,FilmReleaseDate
2,304211,00bSwKf,161966,PlayedFromRecommendation,2017-04-29 07:18:04,2,6,FilmReleaseDate
3,304211,00bSwKf,5617,PlayedFromRecommendation,2017-04-29 07:18:16,0,6,FilmReleaseDate
4,304211,00bSwKf,113829,PlayedFromRecommendation,2017-04-29 07:18:34,7,6,FilmReleaseDate


In [27]:
rec_played_alg2.count()

userId           9142
loginId          9142
movieId          9142
action           9142
tstamp           9142
position         9142
sessionIds       9142
recommenderId    9142
dtype: int64

In [28]:
rec_played_alg2.groupby("action").count()

,userId,loginId,movieId,tstamp,position,sessionIds,recommenderId
action,,,,,,,
PlayedFromNext,1335,1335,1335,1335,1335,1335,1335
PlayedFromPrevious,181,181,181,181,181,181,181
PlayedFromRecommendation,7626,7626,7626,7626,7626,7626,7626


In [29]:
rec_played_alg2.to_csv("../Clean_Data/recs_viewed_sessionid.csv")